<a href="https://colab.research.google.com/github/jsilveira1409/CIVIL-459-Animal-Pose-Estimation/blob/sda-dev/openpifpaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import sys
import openpifpaf
import IPython
from openpifpaf.transforms import Crop
from openpifpaf.plugins.animalpose import AnimalKp
from PIL import Image
import gdown
import subprocess
openpifpaf.show.Canvas.show = True
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from openpifpaf_animalplugin import AnimalPoseEstimation
from openpifpaf_sdaplugin import SDA
import coco_adapt as coco_adapt

# Dataset download and manipulation

In [ ]:
if 'google.colab' in sys.modules:
    !rm -rf plugins
    !rm -rf openpifpaf_animalplugin
    !git clone --branch sda-dev https://github.com/jsilveira1409/CIVIL-459-Animal-Pose-Estimation.git plugins
    !mv plugins/openpifpaf_animalplugin/ openpifpaf_animalplugin
    !mv plugins/coco_adapt.py coco_adapt.py

coco_adapt.download_dataset()
coco_adapt.adapt_to_coco()
coco_adapt.split_data()

from openpifpaf_animalplugin import AnimalPoseEstimation
from openpifpaf_sdaplugin import SDA
config = openpifpaf.plugin.register()
openpifpaf.DATAMODULES

# Native Animal Pose Estimation Plugin

In [ ]:
dataset = AnimalKp()
dataset.batch_size =1
dataloader = dataset.train_loader()

## Train

In [ ]:
cmd = "python3 -m openpifpaf.train \
  --lr=0.0003 \
  --momentum=0.95 \
  --clip-grad-value=10.0\
  --b-scale=10.0 \
  --batch-size=16 \
  --loader-workers=12 \
  --epochs=350 \
  --lr-decay 310 330 \
  --lr-decay-epochs=10 \
  --val-interval 5 \
  --checkpoint=shufflenetv2k30 \
  --lr-warm-up-start-epoch=200 \
  --dataset=animal \
  --animal-square-edge=385 \
  --animal-upsample=2 \
  --animal-bmin=2 --animal-extended-scale \
  --weight-decay=1e-5"

subprocess.call(cmd, shell=True)

## Eval

In [ ]:
cmd = "python3 -m openpifpaf.eval \
    --dataset=animal \
    --checkpoint=shufflenetv2k30-animalpose \
    --batch-size=1 \
    --skip-existing \
    --loader-workers=8 \
    --force-complete-pose \
    --seed-threshold=0.01 --instance-threshold=0.01"

subprocess.call(cmd, shell=True)

# Custom Animal Pose Estimation Dataset and Plugin

In [ ]:
dataset = AnimalPoseEstimation()
dataloader = dataset.train_loader()

# Semantic Data Augmentation

In [ ]:
sda = SDA(output_dir='data-animalpose/sda')
sda(dataset[0][0], dataset[0][1])

# Training

In [ ]:
!python3 -m openpifpaf.train \
    --dataset custom_animal \
    --basenet=shufflenetv2k30 \
    --lr=0.00002 \
    --momentum=0.95 \
    --epochs=50 \
    --lr-decay 160 260 \
    --lr-decay-epochs=10  \
    --weight-decay=1e-5 \
    --weight-decay=1e-5 \
    --val-interval 10 \
    --loader-workers 2 \
    --batch-size 5

# Save Results

In [ ]:
import zipfile
from google.colab import drive, files
import os

# create a zip file of the folder you want to save
folder_path = '/outputs'
zip_path = '/content/myfolder.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            zipf.write(os.path.join(root, file))

# save the zip file to your Google Drive
drive_path = '/content/drive/My Drive/myfolder.zip'
!cp '{zip_path}' '{drive_path}'